<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0703iwashita_yoshihara_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
- tile: `2022_0703iwashita_yoshihara_demo.ipynb'
- data: 2022_0703
- author: 浅川伸一
---

# SNOW の文章をいじってみる

In [ ]:
import torch
try:
    import bit
except ImportError:
    !pip install ipynbname --upgrade > /dev/null 2>&1 
    !git clone https://github.com/ShinAsakawa/bit.git
import bit

isColab = bit.isColab
HOME = bit.HOME
%config InlineBackend.figure_format = 'retina'

import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade openpyxl
    !pip install --upgrade pandas
    !pip install --upgrade fugashi[unidic-lite]
    !pip install --upgrade ipadic
    !python -m unidic download
    !pip install transformers
    !pip install --upgrade jaconv

In [3]:
if isColab:
    # Google Drive からデータを入手
    # このセルを実行するとブラウザの別タブで Google アカウントへの認証が求められる
    # Google アカウントを選択するとクリデンシャルキーが表示されるので，そのキーを
    # コピーして，このセルの出力欄にある空欄に貼り付けてエンターキー (リターンキー) を押下する
    # Import PyDrive and associated libraries.
    # This only needs to be done once per notebook.
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # Authenticate and create the PyDrive client.
    # This only needs to be done once per notebook.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    # 以下実際のデータの情報
    pretrained_models = {
        '2022_0627bert_model.pt': '1oAlxm93un85Y2w2xE92A3i4ZUamma73T',
        '2022_0628bert_model_ja_no_pretrained.pt': '1_J3aymd8-xD8tQtSNkAsoQL0uRIVuk6u',
        '2022_0629bert_model_ja_pretrained.pt':'1ksxLOkj0tO1-DQmKKiLStpx1aNRn17Tp',
        '2022_0630bert_snow_model_no_pretrained': '1OIQwk0RqnFeHj_AmdBK49Ka-aLLgl6SZ',
        '2022_0701bert_snow_model_ja_no_pretrained':'1IMkhjApehc-0R0dSvRPU5HNBzheH7Aks'}
    
    model_name = list(pretrained_models.keys())[0]

    downloaded = drive.CreateFile({'id': pretrained_models[model_name]})
    downloaded.GetContentFile(model_name)
else:
    model_name = '2022_0629bert_snow_pretrained_model.pt'
print(f'model_name:{model_name}')
model = torch.load(model_name)
model.eval();

from transformers import BertJapaneseTokenizer
tknz_model_name = 'cl-tohoku/bert-base-japanese'  # 東北大学乾研による 日本語 BERT 実装
# see https://huggingface.co/sonoisa/sentence-bert-base-ja-mean-tokens-v2
# model_name_ja = 'sonoisa/sentence-bert-base-ja-mean-tokens-v2'  # 東北大学乾研による 日本語 BERT 実装
tknz = BertJapaneseTokenizer.from_pretrained(tknz_model_name)


model_name:2022_0627bert_model.pt


# やさしい日本語のデータを取得する

In [4]:
import os
import sys
import requests
import pandas as pd
import jaconv
SNOWs={'T15': {'url':"https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T15-2020.1.7.xlsx"},
       'T23': {'url':"https://filedn.com/lit4DCIlHwxfS1gj9zcYuDJ/SNOW/T23-2020.1.7.xlsx"},}


for corpus in SNOWs:
    url = SNOWs[corpus]['url']
    excel_fname = corpus + '-2020.1.7.xlsx'
    if not os.path.exists(excel_fname):
        r = requests.get(url)
        with open(excel_fname, 'wb') as f:
            total_length = int(r.headers.get('content-length'))
            print(f'{excel_fname} をダウンロード中 {total_length} バイト')
            f.write(r.content)

    SNOWs[corpus]['df'] = pd.read_excel(excel_fname, engine='openpyxl')
    SNOWs[corpus]['df'] = SNOWs[corpus]['df'].rename(columns={'#日本語(原文)': 'ja', 
                                                              '#やさしい日本語':'easy_ja',
                                                              '#英語(原文)':'en'})

_snow_sents = SNOWs['T15']['df']['easy_ja'].to_list() + SNOWs['T23']['df']['easy_ja'].to_list()
snow_sents = [jaconv.normalize(line, 'NFKC') for line in _snow_sents]
print(snow_sents[:3])

['誰が一番に着くか私には分かりません。', '多くの動物が人間によって殺された。', '私はテニス部員です。']


In [5]:
%%time
snow_encoded = tknz(snow_sents,return_tensors='pt', padding='longest')

CPU times: user 18 s, sys: 359 ms, total: 18.3 s
Wall time: 21.1 s


In [6]:
class SentenceBertJapanese:
    def __init__(self, 
                 model=model,
                 tokenizer=tknz,
                 device=None):
        self.tokenizer = tokenizer
        self.model = model
        self.model.eval()

        if device is None:
            device = "cuda" if torch.cuda.is_available() else "cpu"
        self.device = torch.device(device)
        self.model.to(device)

    def _mean_pooling(self, model_output, attention_mask):
        token_embeddings = model_output[0] #First element of model_output contains all token embeddings
        input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
        return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

    @torch.no_grad()
    def encode(self, sentences, batch_size=8):
        all_embeddings = []
        iterator = range(0, len(sentences), batch_size)
        for batch_idx in iterator:
            batch = sentences[batch_idx:batch_idx + batch_size]

            encoded_input = self.tokenizer.batch_encode_plus(batch, padding="longest", 
                                           truncation=True, return_tensors="pt").to(self.device)
            model_output = self.model(**encoded_input)
            sentence_embeddings = self._mean_pooling(model_output, encoded_input["attention_mask"]).to('cpu')

            all_embeddings.extend(sentence_embeddings)

        # return torch.stack(all_embeddings).numpy()
        return torch.stack(all_embeddings)



In [7]:
import numpy as np
sbert = SentenceBertJapanese()

In [8]:
%%time
# 時間がかかるので最初の 300 文だけ
snow_embeddings = sbert.encode(snow_sents[:300], batch_size=8)
print(f"len(Sentence embeddings):{len(snow_embeddings)}")

len(Sentence embeddings):300
CPU times: user 27.8 s, sys: 198 ms, total: 28 s
Wall time: 29 s


## 1.1 意味が近い文章をクラスタリング

In [10]:
from sklearn.cluster import KMeans

num_clusters = 8  # クラスター数は適当に決めました
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(snow_embeddings)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for _id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(snow_sents[_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['彼女は私たちの世話をしてくれる。', '私たちは国際人になりたいと思います。', '父は私が外国へ行くことを許した。', '彼の小説は1つも読んでいない。', '彼はどこへ行ったらいいか分からなかった。', '彼もそれを見た。', '2時間も待った。', '私は彼女が子供の頃から知っています。', '彼は危険をものともしない。', '彼らはちゃんと私の顔を見られなかった。', 'クラスの少年の数は30人です。', '子供たちを学校に連れて行った。', '彼は前は思いのままにお金を使った。', '私たちはロンドンでお互いを知った。', '彼は何でも、知っているように見せる。', '起きてみると彼女は病院にいた。', '私は彼女の合格を喜んだ。', '私たちは上の方から叫ぶ声がするのを耳にした。', '私の兄は交通事故で死んだ。', '彼は先生の注意を大切にした。', '私はその事実を知っていましたよ。', '彼は事故のため仕事ができなくなった。', '彼はその時までに完全に頭にきていた。', '誰も彼の間違いを指摘するだけの勇気がなかった。', '私は昨日映画を見に行きました。', '彼はあなたの成功を見て自分もそうなりたいと思っている。', '彼は親切にも私に町を案内してくれた。', '彼は私たちのチームで最も価値のある選手です。', '彼女は他のどの学生にも負けず勉強する。', 'ジョンは相手の王を逃げられなくすれば勝つゲームがうまいです。', '彼女は彼に自転車を使うように言った。', '彼は彼女の妹と結婚した。', '彼は言ったことをできずに終わった。', '彼は今日、お弁当を持って来た。', '彼女の声は震えていた。', '彼は新聞に自分の顔を発表したかった。', '彼の父は事業に失敗した。', '彼の話はみんなを楽しませた。', '私たちはその少年に案内を頼んだ。', '彼は自分の考えを書いておいた。', '彼は短い病気の後で死んだ。', '国民は新しい政府を作った。', '私はその本を読み終わったところです。', '私の母は私に鞄を作ってくれました。', '遅れたので私はバスに乗った。', '彼は何も話すことが出来ないほどに腹を立てていた。', '彼はその新しい計画に反対した。', '私はそのゲームはとても面白いと思った。', 

## 1.2 意味が近い文を検索

In [9]:
import scipy.spatial

N = 10
queries = snow_sents[:N]  # 試みに最初の N 文 だけやってみる
query_embeddings = sbert.encode(queries).numpy()

closest_n = 5             # 最近隣文 closest_n だけ表示する
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], snow_embeddings, metric="cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("=" * 78)
    print("Query:", query)
    print("Top 5 most similar sentences:")

    for idx, distance in results[0:closest_n]:
        print(snow_sents[idx].strip(), f"(Score: {distance/2:.4f})")

Query: 誰が一番に着くか私には分かりません。
Top 5 most similar sentences:
誰が一番に着くか私には分かりません。 (Score: 0.0000)
あなたはポーラに何と言ったのですか。 (Score: 0.0080)
私はあなたの言うことが全くわかりません。 (Score: 0.0081)
大阪にはどうやって行くんですか。 (Score: 0.0086)
あなたはそれを持っていますか。 (Score: 0.0088)
Query: 多くの動物が人間によって殺された。
Top 5 most similar sentences:
多くの動物が人間によって殺された。 (Score: 0.0000)
ジョンは相手の王を逃げられなくすれば勝つゲームがうまいです。 (Score: 0.0141)
彼は私にそこへ行けと命令した。 (Score: 0.0147)
彼は何も話すことが出来ないほどに腹を立てていた。 (Score: 0.0149)
私の兄は交通事故で死んだ。 (Score: 0.0149)
Query: 私はテニス部員です。
Top 5 most similar sentences:
私はテニス部員です。 (Score: 0.0000)
ボブは私の友達です。 (Score: 0.0075)
彼は野球の選手です。 (Score: 0.0091)
私たち学生はみんな野球が好きだ。 (Score: 0.0103)
私は音楽が好きではありません。 (Score: 0.0104)
Query: エミは幸せそうに見えます。
Top 5 most similar sentences:
エミは幸せそうに見えます。 (Score: 0.0000)
彼女は元気がなくなった。 (Score: 0.0110)
彼は少し疲れているようですね。 (Score: 0.0115)
私たちは国際人になりたいと思います。 (Score: 0.0116)
彼女は大変忙しかった。 (Score: 0.0117)
Query: この事実を覚えておいてください。
Top 5 most similar sentences:
この事実を覚えておいてください。 (Score: 0.0000)
私はその事実を知っていましたよ。 (Score: 0.0076)
月曜日までにこの仕事を終わら

## 1.3 文の意味関係の可視化 tSNE による散布図

以下は colab ではメモリ不足で動作しません。
後日公開させていただきます。

In [ ]:
!git clone https://github.com/ShinAsakawa/ccap.git
try:
    import japanize_matplotlib
except ImportError:
    !pip install japanize_matplotlib

if isColab:
    #!pip install mecab-python3 unidic-lite
    #https://colab.research.google.com/github/himkt/konoha/blob/master/example/Konoha_Example.ipynb#scrollTo=wBxBPzHtYYXQ
    !sudo apt update -qy && apt install -qy mecab libmecab-dev mecab-ipadic-utf8 > /cev/null
    !pip install -q konoha[all]  

    # 形態素分析ライブラリーMeCab と 辞書(mecab-ipadic-NEologd)のインストール 
    # https://qiita.com/jun40vn/items/78e33e29dce3d50c2df1
    !apt-get -q -y install sudo file mecab libmecab-dev mecab-ipadic-utf8 git curl python-mecab > /dev/null
    !git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 
    !echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n > /dev/null 2>&1
    !pip install mecab-python3 > /dev/null

    !ln -s /etc/mecabrc /usr/local/etc/mecabrc

In [ ]:
from ccap import tsne
X = tsne.tsne(snow_embeddings.clone().numpy())

Preprocessing the data using PCA...
